### BP to LF transition:
#### Variables
For each $e\in E$ and every time $t$:  
-  $requested(e,t)$        
-  $blocked(e,t)$  
- $triggered(e,t)$  



For each state $s\in S(bt)$ of every b-thread $bt$ and every time $t$:

    
- $at(s,t)$  

_________________________________________________________________
#### Rules

$\forall t \colon  \oplus_{e\in E}(triggered(e,t))$

$\forall t,e \colon  triggered(e,t) \Rightarrow requested(e,t) \wedge \neg blocked(e,t)$

$\forall bt, t \colon  \oplus_{s\in S(bt)}(at(s,t))$


$at(s,t) \iff ( t=0 \wedge s \in INITIAL)  \vee (at(s',t-1) \wedge triggered(e,t-1) \wedge (s',e,s)\in TRANSITIONS)$

$requested(e,t) \iff \bigvee_{s \in S} e \in REQUESTED(s) \wedge at(s,t)$

$blocked(e,t) \iff \bigvee_{s \in S} e \in BLOCKED(s) \wedge at(s,t)$


In [55]:
import os
import re
import yaml
from yaml import CLoader as Loader, CDumper as Dumper
from collections import defaultdict

### Parse BThreads yaml

In [11]:
r = re.compile("bt\d.yaml")
bthreads_files = list(filter(r.match, os.listdir('.')))
bthreads_files

['bt1.yaml', 'bt3.yaml', 'bt2.yaml']

In [208]:
bt_dict = dict()
bthreads = dict()
for bt_file in bthreads_files:
    with open(bt_file, 'r') as bt_yaml:
        bt_dict[bt_file.split('.')[0]] = yaml.load(bt_yaml, Loader=Loader)
# bt_dict

In [209]:
INITIALS = set()
EVENTS = set()
STATES = set()
BLOCKED = defaultdict(list)
REQUESTED = defaultdict(list)
TIME = [i for i in range(3)]
TRANSITIONS = set()
for bt in bt_dict:
    INITIALS.add(f"{bt}_{bt_dict[bt]['initial']}")
    for step in bt_dict[bt]['states']:
        step_id = f"{bt}_{step['id']}"
        for key, values in step.items():
            if key == 'id':
                STATES.add(step_id)
            if key in ('R', 'W', 'B'):
                for val in values:
                    EVENTS.add(val)
              
        if len(step['B']) > 0:
            BLOCKED[step_id].extend(step['B'])
        if len(step['R']) > 0:
            REQUESTED[step_id].extend(step['R'])
    for t in bt_dict[bt]['transitions']:
        TRANSITIONS.add((f"{bt}_state_{t['source']}",f"{bt}_state_{t['target']}",t['event']))

In [210]:
INITIALS

{'bt1_state_0', 'bt2_state_0', 'bt3_state_0'}

In [211]:
STATES 

{'bt1_state_0',
 'bt1_state_1',
 'bt1_state_2',
 'bt1_state_3',
 'bt2_state_0',
 'bt2_state_2',
 'bt2_state_3',
 'bt3_state_0',
 'bt3_state_1',
 'bt3_state_2',
 'bt3_state_3'}

In [212]:
EVENTS 

{'g1', 'g2', 'g3', 'h1', 'h2', 'h3', 'o1', 'o2', 'o3'}

In [213]:
BLOCKED 

defaultdict(list,
            {'bt3_state_1': ['o1'],
             'bt3_state_2': ['o3'],
             'bt2_state_0': ['o2'],
             'bt2_state_2': ['o1'],
             'bt2_state_3': ['o3']})

In [214]:
REQUESTED

defaultdict(list,
            {'bt1_state_0': ['g1', 'g2', 'g3'],
             'bt1_state_1': ['o1', 'o2', 'o3'],
             'bt1_state_2': ['h1', 'h2', 'h3']})

In [215]:
TIME

[0, 1, 2]

In [216]:
TRANSITIONS

{('bt1_state_0', 'bt1_state_1', 'g1'),
 ('bt1_state_0', 'bt1_state_1', 'g2'),
 ('bt1_state_0', 'bt1_state_1', 'g3'),
 ('bt1_state_1', 'bt1_state_3', 'o1'),
 ('bt1_state_1', 'bt1_state_3', 'o2'),
 ('bt1_state_1', 'bt1_state_3', 'o3'),
 ('bt1_state_2', 'bt1_state_0', 'h1'),
 ('bt1_state_2', 'bt1_state_0', 'h2'),
 ('bt1_state_2', 'bt1_state_0', 'h3'),
 ('bt2_state_0', 'bt2_state_1', 'h2'),
 ('bt2_state_0', 'bt2_state_2', 'h1'),
 ('bt2_state_0', 'bt2_state_3', 'h3'),
 ('bt3_state_0', 'bt3_state_1', 'g1'),
 ('bt3_state_0', 'bt3_state_2', 'g3'),
 ('bt3_state_0', 'bt3_state_3', 'g2')}

### Literals creation

In [172]:
import aiger

In [217]:
atoms = dict()
def get_atom(name):
    if name not in atoms:
        atoms[name] = aiger.atom(name)
    return atoms[name]
        

1. $\forall t \colon  \oplus_{e\in E}(triggered(e,t))$

In [218]:
def unique_event(events=EVENTS , time=TIME):
    triggered = [get_atom(f'triggered_{e}_at_{t}') for e in events for t in time]
    formula = False

    for t in triggered:
        c = t
        for tt in triggered:
            if t != tt:
                c = c & ~tt
        formula = c | formula
    return formula
  

2. $\forall t,e \colon  triggered(e,t) \Rightarrow requested(e,t) \wedge \neg blocked(e,t)$
 

In [175]:
def bp(events=EVENTS , time=TIME):
    literals = [(get_atom(f'requested_{e}_at_{t}'),
                  get_atom(f'blocked_{e}_at_{t}'),
                  get_atom(f'triggered_{e}_at_{t}')) for e in events for t in time]
    formula = True
    for r,b,t in literals:
        formula = t.implies(r & ~b) & formula
        
    return formula

3. $\forall bt, t \colon  \oplus_{s\in S(bt)}(at(s,t))$  

In [188]:
def one_state_at_time(states=STATES , time=TIME):
    literals = [get_atom(f'{s}_at_{t}') for s in states for t in time]
    formula = False

    for l in literals:
        c = l
        for ll in literals:
            if l != ll:
                c = c & ~ll
        formula = c | formula
    return formula

4. $at(s,t) \iff ( t=0 \wedge s \in INITIAL)  \vee (at(s',t-1) \wedge triggered(e,t-1) \wedge (s',e,s)\in TRANSITIONS)$  

In [206]:
def check_transitions(states=STATES , time=TIME, initials=INITIALS, transitions=TRANSITIONS):
    literals = [(get_atom(f'requested_{e}_at_{t}'),
              get_atom(f'triggered_{e}_at_{t}'),
                 
                 
                 for e in events for t in time]
    formula = True
    for r,b,t in literals:
        formula = t.implies(r & ~b) & formula
        
    return formula

5. $requested(e,t) \iff \bigvee_{s \in S} e \in REQUESTED(s) \wedge at(s,t)$

In [230]:
def requested(events=EVENTS, states=STATES , time=TIME):
    literals = [(get_atom(f'requested_{e}_at_{t}'),
              get_atom(f'blocked_{e}_at_{t}'),
              get_atom(f'triggered_{e}_at_{t}')) for e in events for t in time]

6. $blocked(e,t) \iff \bigvee_{s \in S} e \in BLOCKED(s) \wedge at(s,t)$  

In [ ]:
def blocked():
    pass

In [176]:
formula = (unique_event() &bp())
cnf = CNF(from_aiger=(formula.aig))
cnf.to_file('cnf_bp.cnf')
cnf2 = aig2cnf(unique_event().aig)


In [177]:
len(cnf2.clauses)

81